In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# To run cross validation parallelized
from joblib import Parallel, delayed

2024-06-17 15:13:10.030396: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 15:13:10.032807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 15:13:10.082871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 15:13:10.084029: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-17 15:13:10.884049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate_parallelized(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
    n_jobs=-1,  # Set the number of parallel jobs, -1 for using all available cores
):
    def process_fold(train_index, test_index, dates, cement_types, x, y):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        set_seeds(SEED + REPEAT)

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        return score

    scores = Parallel(n_jobs=n_jobs)(
        delayed(process_fold)(train_index, test_index, dates, cement_types, x, y)
        for train_index, test_index in cv.split(x)
    )

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]

    results = [scores_final]
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class GRU(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.GRU(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
SEED = 47
REPEAT = 0

<IPython.core.display.Javascript object>

In [14]:
def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [15]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [16]:
def suppress_warnings():
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

<IPython.core.display.Javascript object>

In [17]:
def supress_other_stuff():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    # os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

<IPython.core.display.Javascript object>

In [18]:
suppress_warnings()
supress_other_stuff()

<IPython.core.display.Javascript object>

In [19]:
index_to_save = 7

<IPython.core.display.Javascript object>

In [20]:
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = ["Cement_Type_CP III-32"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [21]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "209",
    "Plant": "E",
    "Features": "Chemical + CS3 + CS7",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "GRU",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [22]:
df = pd.read_csv("../../../../../../data/processed/209/e.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [23]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "Blaine",
        
        
		
        "Final setting time",
        "Initial setting time",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [24]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [25]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - GRU

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [26]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate_parallelized(
        GRU,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.457 (0.045)
MAE: 1.110 (0.031)
MAPE: 0.025 (0.001)
R2: 0.769 (0.016)


******
[TEST]
******
RMSE: 1.570 (0.106)
MAE: 1.193 (0.075)
MAPE: 0.027 (0.002)
R2: 0.727 (0.042)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.488 (0.223)
MAE: 1.191 (0.207)
MAPE: 0.027 (0.004)
R2: 0.754 (0.072)


******
[TEST]
******
RMSE: 1.782 (0.209)
MAE: 1.389 (0.185)
MAPE: 0.031 (0.004)
R2: 0.622 (0.077)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.480 (0.210)
MAE: 1.178 (0.203)
MAPE: 0.026 (0.004)
R2: 0.754 (0.068)


******
[TEST]
******
RMSE: 1.767 (0.157)
MAE: 1.406 (0.135)
MAPE: 0.032 (0.003)
R2: 0.541 (0.126)


Minutes Elapsed:  7.38828110297521


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model Cross Validation Timesteps RMSE Test            \
                                                              mean       std   
0  Chemical + CS3 + CS7   GRU   Repeated KFold         1  1.569527  0.106085   
1  Chemical + CS3 + CS7   GRU   Repeated KFold         7  1.782054  0.209259   
2  Chemical + CS3 + CS7   GRU   Repeated KFold        14  1.766502  0.157279   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  1.193480  0.074757  0.027055  0.001698  0.727063  0.042092  
1  1.388751  0.184714  0.031384  0.004208  0.622319  0.076694  
2  1.405567  0.134680  0.031568  0.003020  0.541081  0.125720

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [28]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for repeat in range(repeats):
        REPEAT = repeat
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate_parallelized(
            GRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.974 (0.517)
MAE: 2.368 (0.395)
MAPE: 0.054 (0.010)
R2: -0.704 (1.135)


******
[TEST]
******
RMSE: 5.487 (2.401)
MAE: 4.097 (1.384)
MAPE: 0.093 (0.030)
R2: -22.303 (31.376)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.487 (0.395)
MAE: 1.211 (0.386)
MAPE: 0.028 (0.009)
R2: 0.595 (0.217)


******
[TEST]
******
RMSE: 6.536 (2.165)
MAE: 5.376 (1.894)
MAPE: 0.121 (0.042)
R2: -25.417 (27.574)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.401 (0.288)
MAE: 1.135 (0.256)
MAPE: 0.026 (0.006)
R2: 0.581 (0.336)


******
[TEST]
******
RMSE: 5.709 (2.661)
MAE: 4.926 (2.638)
MAPE: 0.110 (0.058)
R2: -26.269 (39.800)


Minutes Elapsed:  2.6708969990412395


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation Timesteps RMSE Test  \
                                                                         mean   
0  Chemical + CS3 + CS7   GRU  Blocking Time Series Split         1  5.487451   
1  Chemical + CS3 + CS7   GRU  Blocking Time Series Split         7  6.535927   
2  Chemical + CS3 + CS7   GRU  Blocking Time Series Split        14  5.709400   
3  Chemical + CS3 + CS7   GRU              Repeated KFold         1  1.569527   
4  Chemical + CS3 + CS7   GRU              Repeated KFold         7  1.782054   
5  Chemical + CS3 + CS7   GRU              Repeated KFold        14  1.766502   

             MAE Test           MAPE Test              R2 Test             
        std      mean       std      mean       std       mean        std  
0  2.400799  4.097112  1.384146  0.092938  0.030276 -22.302986  31.376089  
1  2.164737  5.375605  1.894320  0.121495  0.042159 -25.417392  27.574363  
2  2.660521  4.925737  2.637598  0.109836  0.057885 -26.269426  39.800225  
3  0.106085  1.193480  0.074757  0.027055  0.001698   0.727063   0.042092  
4  0.209259  1.388751  0.184714  0.031384  0.004208   0.622319   0.076694  
5  0.157279  1.405567  0.134680  0.031568  0.003020   0.541081   0.125720

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [30]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for repeat in range(repeats):
        REPEAT = repeat
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate_parallelized(
            GRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.664 (0.249)
MAE: 1.285 (0.233)
MAPE: 0.028 (0.005)
R2: 0.401 (0.695)


******
[TEST]
******
RMSE: 8.764 (13.111)
MAE: 5.310 (7.125)
MAPE: 0.127 (0.173)
R2: -29.247 (59.083)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.274 (0.215)
MAE: 1.004 (0.190)
MAPE: 0.022 (0.004)
R2: 0.753 (0.140)


******
[TEST]
******
RMSE: 8.219 (11.042)
MAE: 5.386 (6.414)
MAPE: 0.129 (0.156)
R2: -20.791 (43.203)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.251 (0.111)
MAE: 0.981 (0.091)
MAPE: 0.022 (0.002)
R2: 0.730 (0.210)


******
[TEST]
******
RMSE: 4.815 (3.852)
MAE: 3.434 (2.189)
MAPE: 0.082 (0.055)
R2: -3.049 (6.548)


Minutes Elapsed:  13.37274667819341


<IPython.core.display.Javascript object>

In [31]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation Timesteps RMSE Test  \
                                                                         mean   
0  Chemical + CS3 + CS7   GRU  Blocking Time Series Split         1  5.487451   
1  Chemical + CS3 + CS7   GRU  Blocking Time Series Split         7  6.535927   
2  Chemical + CS3 + CS7   GRU  Blocking Time Series Split        14  5.709400   
3  Chemical + CS3 + CS7   GRU              Repeated KFold         1  1.569527   
4  Chemical + CS3 + CS7   GRU              Repeated KFold         7  1.782054   
5  Chemical + CS3 + CS7   GRU              Repeated KFold        14  1.766502   
6  Chemical + CS3 + CS7   GRU           Time Series Split         1  8.763889   
7  Chemical + CS3 + CS7   GRU           Time Series Split         7  8.219483   
8  Chemical + CS3 + CS7   GRU           Time Series Split        14  4.815077   

              MAE Test           MAPE Test              R2 Test             
         std      mean       std      mean       std       mean        std  
0   2.400799  4.097112  1.384146  0.092938  0.030276 -22.302986  31.376089  
1   2.164737  5.375605  1.894320  0.121495  0.042159 -25.417392  27.574363  
2   2.660521  4.925737  2.637598  0.109836  0.057885 -26.269426  39.800225  
3   0.106085  1.193480  0.074757  0.027055  0.001698   0.727063   0.042092  
4   0.209259  1.388751  0.184714  0.031384  0.004208   0.622319   0.076694  
5   0.157279  1.405567  0.134680  0.031568  0.003020   0.541081   0.125720  
6  13.110577  5.309655  7.125006  0.127482  0.173447 -29.246866  59.082704  
7  11.041810  5.385530  6.414281  0.129005  0.156199 -20.791258  43.203270  
8   3.852047  3.433667  2.189301  0.081982  0.055453  -3.048583   6.547770

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [32]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()

    def parallel_repeats(repeat):
        set_seeds(SEED + repeat)

        scores_final = None

        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1).copy()
        y = df_copy["CS28"].copy()

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", GRU())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

        return scores_final

    scores = Parallel(n_jobs=5)(
        delayed(parallel_repeats)(repeat) for repeat in range(repeats)
    )
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += value

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.518 (0.042)
MAE: 1.160 (0.047)
MAPE: 0.026 (0.001)
R2: 0.782 (0.012)


******
[TEST]
******
RMSE: 1.929 (0.303)
MAE: 1.501 (0.251)
MAPE: 0.034 (0.006)
R2: -0.096 (0.343)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.624 (0.269)
MAE: 1.340 (0.229)
MAPE: 0.030 (0.005)
R2: 0.744 (0.082)


******
[TEST]
******
RMSE: 2.140 (0.125)
MAE: 1.731 (0.064)
MAPE: 0.039 (0.001)
R2: -0.347 (0.157)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.320 (0.121)
MAE: 1.037 (0.094)
MAPE: 0.023 (0.002)
R2: 0.834 (0.030)


******
[TEST]
******
RMSE: 2.157 (0.204)
MAE: 1.728 (0.211)
MAPE: 0.039 (0.005)
R2: -0.505 (0.291)


Minutes Elapsed:  4.654858692487081


<IPython.core.display.Javascript object>

In [33]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation Timesteps  \
                                                                       
0   Chemical + CS3 + CS7   GRU  Blocking Time Series Split         1   
1   Chemical + CS3 + CS7   GRU  Blocking Time Series Split         7   
2   Chemical + CS3 + CS7   GRU  Blocking Time Series Split        14   
3   Chemical + CS3 + CS7   GRU           Out of time Split         1   
4   Chemical + CS3 + CS7   GRU           Out of time Split         7   
5   Chemical + CS3 + CS7   GRU           Out of time Split        14   
6   Chemical + CS3 + CS7   GRU              Repeated KFold         1   
7   Chemical + CS3 + CS7   GRU              Repeated KFold         7   
8   Chemical + CS3 + CS7   GRU              Repeated KFold        14   
9   Chemical + CS3 + CS7   GRU           Time Series Split         1   
10  Chemical + CS3 + CS7   GRU           Time Series Split         7   
11  Chemical + CS3 + CS7   GRU           Time Series Split        14   

   RMSE Test             MAE Test           MAPE Test              R2 Test  \
        mean        std      mean       std      mean       std       mean   
0   5.487451   2.400799  4.097112  1.384146  0.092938  0.030276 -22.302986   
1   6.535927   2.164737  5.375605  1.894320  0.121495  0.042159 -25.417392   
2   5.709400   2.660521  4.925737  2.637598  0.109836  0.057885 -26.269426   
3   1.928614   0.302941  1.500755  0.250622  0.034339  0.005646  -0.096111   
4   2.140498   0.125465  1.731451  0.063853  0.039443  0.001285  -0.346863   
5   2.157021   0.203998  1.727787  0.211040  0.039299  0.004675  -0.505055   
6   1.569527   0.106085  1.193480  0.074757  0.027055  0.001698   0.727063   
7   1.782054   0.209259  1.388751  0.184714  0.031384  0.004208   0.622319   
8   1.766502   0.157279  1.405567  0.134680  0.031568  0.003020   0.541081   
9   8.763889  13.110577  5.309655  7.125006  0.127482  0.173447 -29.246866   
10  8.219483  11.041810  5.385530  6.414281  0.129005  0.156199 -20.791258   
11  4.815077   3.852047  3.433667  2.189301  0.081982  0.055453  -3.048583   

               
          std  
0   31.376089  
1   27.574363  
2   39.800225  
3    0.342934  
4    0.156963  
5    0.290706  
6    0.042092  
7    0.076694  
8    0.125720  
9   59.082704  
10  43.203270  
11   6.547770

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [34]:
path = "../../../../../../reports/results/local_models/209/e/full/"
filename = f"gru_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [35]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/209/e/grouped/"
filename = f"gru_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>